In [104]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
from torch import optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression

In [105]:
X,Y,coef= make_regression(n_samples=1000,n_features=16,n_targets=1,noise=1, coef=True)

In [106]:
Y= Y.reshape(-1,1)

In [107]:
X.shape,Y.shape,coef.shape

((1000, 16), (1000, 1), (16,))

# 机器学习方法

In [108]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [109]:
reg = LinearRegression()
reg.fit(X,Y)

LinearRegression()

In [110]:
mean_squared_error(reg.predict(X),Y)

1.0067095827403914

# 用来做随机batch的模块

In [111]:
class DiabetesDataset(Dataset):
    def __init__(self, x_, y_):
        self.len = x_.shape[0]
        self.x_data = torch.from_numpy(x_)
        # 这部很关键， 特征类型要求float类型
        self.x_data = self.x_data.float()

        self.y_data = torch.from_numpy(y_)
        self.y_data = self.y_data.float()

    def __getitem__(self, item):
        return self.x_data[item], self.y_data[item]

    def __len__(self):
        return self.len

In [112]:
dataset = DiabetesDataset(X, Y)
train_loader = DataLoader(dataset=dataset, batch_size=100, shuffle=True)

# 网络结构

In [113]:
#网络结构
class NeuralNet(nn.Module):
    def __init__(self, input_size_, hidden_size_, num_classes_):
        super().__init__()
        self.fc1 = nn.Linear(input_size_, hidden_size_) 
        self.fc2 = nn.Linear(hidden_size_, num_classes_)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

# 建立模型训练

In [114]:
model = NeuralNet(16, 8, 1)
#损失函数
criterion = nn.MSELoss(reduction='mean')
#优化算法
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [115]:
total_step = len(train_loader)
num_epochs = 1000
for epoch in range(num_epochs):
    for i, (x, y) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        x = x.reshape(-1, 16)
        # Forward pass
        outputs = model(x)
        loss = criterion(outputs, y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #输出中间信息
        if i % 50 == 0 and epoch%20==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/1000], Step [1/10], Loss: 31712.3809
Epoch [21/1000], Step [1/10], Loss: 7968.8398
Epoch [41/1000], Step [1/10], Loss: 615.2706
Epoch [61/1000], Step [1/10], Loss: 410.2522
Epoch [81/1000], Step [1/10], Loss: 340.3250
Epoch [101/1000], Step [1/10], Loss: 264.9956
Epoch [121/1000], Step [1/10], Loss: 159.2020
Epoch [141/1000], Step [1/10], Loss: 110.9754
Epoch [161/1000], Step [1/10], Loss: 83.8748
Epoch [181/1000], Step [1/10], Loss: 56.4725
Epoch [201/1000], Step [1/10], Loss: 21.0028
Epoch [221/1000], Step [1/10], Loss: 13.6329
Epoch [241/1000], Step [1/10], Loss: 8.1904
Epoch [261/1000], Step [1/10], Loss: 3.6308
Epoch [281/1000], Step [1/10], Loss: 4.0657
Epoch [301/1000], Step [1/10], Loss: 2.4795
Epoch [321/1000], Step [1/10], Loss: 2.4859
Epoch [341/1000], Step [1/10], Loss: 1.8292
Epoch [361/1000], Step [1/10], Loss: 1.1391
Epoch [381/1000], Step [1/10], Loss: 1.1895
Epoch [401/1000], Step [1/10], Loss: 1.2428
Epoch [421/1000], Step [1/10], Loss: 1.0511
Epoch [441/1000

# 保存模型与运行模型

In [116]:
# 保存模型
path = './a.pth'
torch.save(model.state_dict(), path)

In [117]:
# 查看参数
pargram = torch.load('./a.pth')
pargram

OrderedDict([('fc1.weight',
              tensor([[-3.6667, -5.3661, -0.7674, -0.4398, -4.8885,  0.4457, -4.2127, -5.0262,
                        0.1182, -5.1086, -0.5638, -0.5699, -3.2498, -4.6512, -0.7648, -0.9245],
                      [ 2.5073,  4.6040,  0.1939,  0.4286,  3.2740, -0.1376,  2.4669,  3.5374,
                        0.3028,  3.0339,  0.1359,  0.4917,  2.3221,  4.0489, -0.0877,  0.2348],
                      [-4.0278, -5.9198,  0.7995, -0.6522, -5.1610, -0.5550, -3.8845, -6.1311,
                        0.7170, -5.0085,  0.1621, -0.2133, -4.4276, -5.2549,  0.0244,  0.6776],
                      [ 3.0983,  4.1505, -0.6430,  0.7187,  3.9150,  0.5370,  2.6579,  4.4683,
                       -0.4706,  3.7164, -0.2812, -0.0297,  3.3104,  3.7315, -0.0520, -0.5609],
                      [ 3.1253,  4.3618, -0.3414,  0.0862,  3.8483,  0.2011,  3.3958,  4.8006,
                       -0.3718,  3.6805,  0.0551,  0.5733,  3.0807,  3.8930, -0.0916, -0.2089],
                 

In [118]:
pargram['fc1.weight']

tensor([[-3.6667, -5.3661, -0.7674, -0.4398, -4.8885,  0.4457, -4.2127, -5.0262,
          0.1182, -5.1086, -0.5638, -0.5699, -3.2498, -4.6512, -0.7648, -0.9245],
        [ 2.5073,  4.6040,  0.1939,  0.4286,  3.2740, -0.1376,  2.4669,  3.5374,
          0.3028,  3.0339,  0.1359,  0.4917,  2.3221,  4.0489, -0.0877,  0.2348],
        [-4.0278, -5.9198,  0.7995, -0.6522, -5.1610, -0.5550, -3.8845, -6.1311,
          0.7170, -5.0085,  0.1621, -0.2133, -4.4276, -5.2549,  0.0244,  0.6776],
        [ 3.0983,  4.1505, -0.6430,  0.7187,  3.9150,  0.5370,  2.6579,  4.4683,
         -0.4706,  3.7164, -0.2812, -0.0297,  3.3104,  3.7315, -0.0520, -0.5609],
        [ 3.1253,  4.3618, -0.3414,  0.0862,  3.8483,  0.2011,  3.3958,  4.8006,
         -0.3718,  3.6805,  0.0551,  0.5733,  3.0807,  3.8930, -0.0916, -0.2089],
        [ 2.8026,  4.1805,  0.6640,  0.4065,  3.8707, -0.3709,  3.3509,  3.8322,
         -0.1799,  3.9397,  0.3503,  0.3438,  2.4278,  3.6503,  0.6194,  0.7014],
        [ 2.2202,  4.1

In [119]:
# 加载模型
model = NeuralNet(16, 8, 1)
model.load_state_dict(torch.load('./a.pth'))
#必须调用model.eval()将dropout和batch normalization层设置为测试模式，不然会导致不一致的结果。
model.eval()

NeuralNet(
  (fc1): Linear(in_features=16, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=1, bias=True)
)

In [120]:
#预测
predicted = model(torch.from_numpy(X).float())

In [121]:
mean_squared_error(predicted.detach().numpy(),Y)

1.0164083893607927